In [ ]:
import pandas as pd
import os
import sys
sys.path.append("src/")
from utils.processed_data_path import load_processed_data, folder_to_dataframe
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_path = '../data'
processed_data_path = os.path.join(data_path,"processed")
augmented_data_path = os.path.join(data_path,"augmented")

In [ ]:
folder_dict = load_processed_data(processed_data_path,augmented_data_path)
zone_df = folder_to_dataframe(folder_dict)

In [ ]:
print("Dataframe:")
print(zone_df.head(1))
print("\nShape:")
print(zone_df.shape)
print("\nNull values:")
print(zone_df.isnull().sum())
print("\nDataframe description:")
print(zone_df.describe())

In [ ]:
x = ["no-damage", "minor-damage", "mayor-damage", "destroyed"]
print(zone_df[x])
zone_df[x].hist(figsize=(10, 6), bins=10, alpha=0.7)
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()
y = zone_df[x].sum()
plt.bar(x, y)
plt.yscale('log')
plt.show()
correlation_matrix = zone_df[x].corr()
# Crea el mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Calor de Correlación')
plt.show()

- Identificación de posibles relaciones entre variables mediante diagramas de dispersión.
- Visualización de la distribución de una variable numérica mediante un diagrama de caja (boxplot).

# Data analysis & preprocessing


# Creación de mascaras para entrenamiento de segmentación de instancias.


## Pre- & Post- disaster image pair list creation


## Target's polygons visualization

#### label Json file format:


In [ ]:
"""
"features": {
                "lng_lat": [
                    {
                        "properties": {
                            "feature_type": "building", 
                            "subtype": "no-damage", 
                            "uid": "486b0813-ecd2-4b84-856c-9c0e42156953"
                            },
                        "wkt": "POLYGON ((-90.8154467949004 14.39086318334907, -90.81590706308027 14.39071123556949, -90.81587964154231 14.39059650626618, -90.81595559688809 14.3905736709202, -90.8159334443047 14.39048145754321, -90.81586635209149 14.39049581582651, -90.81584174451078 14.39043032648, -90.81537467349251 14.39060467857228, -90.8154467949004 14.39086318334907))"
                    }, 
                    ]
            }, 
            "metadata": {
                "sensor": "WORLDVIEW03_VNIR", 
                "provider_asset_type": "WORLDVIEW03_VNIR", 
                "gsd": 1.3459189, 
                "capture_date": "2018-02-05T17:10:18.000Z", 
                "off_nadir_angle": 17.933279, 
                "pan_resolution": 0.3362646, 
                "sun_azimuth": 150.80763000000002, 
                "sun_elevation": 55.506645, 
                "target_azimuth": 118.24727, 
                "disaster": "guatemala-volcano", 
                "disaster_type": "volcano", 
                "catalog_id": "10400100377A9B00", 
                "original_width": 1024, 
                "original_height": 1024, 
                "width": 1024, 
                "height": 1024, 
                "id": "MjUxMzEyMA.FwIEh3iVRb5-R_EXW9amgViNSoA", 
                "img_name": "guatemala-volcano_00000000_pre_disaster.png"
            }
"""

In [ ]:
import json
import matplotlib.pyplot as plt
from shapely.wkt import loads
from pyproj import Proj
import numpy as np
from utils.processed_data_path import DisasterZoneFolder

folder: DisasterZoneFolder = folder_dict['aug_0_mexico-earthquake_00000003']
plt.figure(figsize=(5, 5))

n_clusters = 0
clusters: np.array
with open(os.path.join(folder.get_folder_path(), folder.get_pre_json()), 'r') as f:
    disaster_json = json.load(f)
    n_clusters = len(disaster_json['features']['lng_lat'])
    for i, building in enumerate(disaster_json['features']['lng_lat']):
        poly = loads(building['wkt'])
        x, y = poly.exterior.xy
        x, y = Proj(proj='merc')(x, y)
        plt.plot(x, y)

plt.title(f"Clusters: {n_clusters}")
plt.show()

## Creation of a new masks for pre-disaster images


## Creation of bounding boxes for post-disaster images


## Imbalance dataset Analysis


## Ground Truth visualization


In [ ]:
from utils.processed_data_path import DisasterZoneFolder
import typing
folder_dict : typing.List[DisasterZoneFolder] = load_processed_data(processed_data_path,augmented_data_path)
zone_df = folder_to_dataframe(folder_dict)

In [46]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def supp(img, target):
    img_disaster = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    target[target[:, :, 0] == 1] = [70, 70, 70]
    target[target[:, :, 0] == 2] = [178, 255, 102]
    target[target[:, :, 0] == 3] = [194, 255, 94]
    target[target[:, :, 0] == 4] = [255, 139, 51]
    target[target[:, :, 0] == 5] = [255, 19, 19]

    alpha = 1  # Ajustar la transparencia de la segunda imagen
    superposed_image = cv2.addWeighted(img_disaster, 0.8, target, alpha, 0)
    return superposed_image


def visualize_bounding_boxes(ax, bounding_boxes):
    damaged = bounding_boxes[bounding_boxes["label"] != "no-damage"]
    # Dibujar cada bounding box y etiqueta en la imagen
    for i in range(len(damaged)):
        row = damaged.iloc[i]
        x, y = (row["x1"], row["y1"])
        z, w = (row["x2"], row["y2"])
        rect = patches.Rectangle(
            (x, y), z-x, w-y, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(z, w, row["label"], color='white', fontsize=8,
                fontweight=750, fontfamily='sans-serif',
                verticalalignment='top',
                bbox=dict(facecolor='red', alpha=0.3,
                          edgecolor='none', pad=3,))


def showImage(img_pre, img_post, target_pre, target_post, bounding_boxes):
    fig, axes = plt.subplots(1, 2, figsize=(22, 22))

    target_pre_img = supp(img_pre, target_pre)
    target_post_img = supp(img_post, target_post)
    # Mostrar las imágenes

    axes[0].imshow(cv2.cvtColor(target_pre_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Mascaras de edificios antes del desastre',
                      fontsize=17, fontweight=800)
    axes[0].axis('off')
    axes[0].text(400, 20, '(Los colores son aleatorios)',
                 fontweight=400, bbox=dict(facecolor='white'))

    axes[1].imshow(target_post_img)
    visualize_bounding_boxes(axes[1], bounding_boxes)
    axes[1].set_title('Edificios dañados después del desastre',
                      fontsize=17, fontweight=800)
    axes[1].axis('off')

    plt.subplots_adjust(wspace=0.0)
    plt.show()

In [1]:

processed_dataset = "../data/processed_dataset"
folder : DisasterZoneFolder;
for folder in folder_dict.values():

    # Cargar las imágenes
    p = os.path.join(folder.get_folder_path(),folder.get_pre());
    img_pre_disaster = cv2.imread(p)
    p = os.path.join(folder.get_folder_path(),folder.get_post());
    img_post_disaster = cv2.imread(p)

    # Cargar los targets
    p = os.path.join(folder.get_folder_path(),folder.get_instance_mask());
    target_pre_disaster_img = cv2.imread(p)
    p = os.path.join(folder.get_folder_path(),folder.get_class_mask());
    target_post_disaster_img = cv2.imread(p)

    # Bounding box
    p = os.path.join(folder.get_folder_path(),folder.get_bbox());
    bounding_boxes = pd.read_csv(p)  # Lista de bounding boxes (x, y, width, height)

    showImage(img_pre_disaster, img_post_disaster,
              target_pre_disaster_img, target_post_disaster_img, bounding_boxes)

    # Cargar las anotaciones de segmentación de los archivos JSON
    # with open(os.path.join(label_dir, pair.get_pre().get_json_file_name()), 'r') as f:
    #    label_pre_disaster_data = json.load(f)
    # with open(os.path.join(label_dir, pair.get_post().get_json_file_name()), 'r') as f:
    #    label_post_disaster_data = json.load(f)

NameError: name 'DisasterZoneFolder' is not defined